In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import torch
from tqdm import tqdm  # For progress bar

c:\Users\aravi\miniconda3\envs\anlp-project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import load_dataset
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from tqdm import tqdm

# Load dataset
# dataset = load_dataset("Vivacem/MMIQC")
# questions = dataset['train']['instruction']
# answers = dataset['train']['output']

# Initialize embedding model
embedding_model_name = "math-similarity/Bert-MLM_arXiv-MP-class_zbMath"
embeddings_model = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={"trust_remote_code": True}
)

# Initialize Chroma
persist_directory = "MMIQC_ChromaDB"
vectorstore = Chroma(
    embedding_function=embeddings_model,
    persist_directory=persist_directory,
)

# # Prepare documents
# documents = [{"id": str(idx), "content": q, "metadata": {"answer": answers[idx]}} 
#             for idx, q in enumerate(questions)]

# # Process in batches
# BATCH_SIZE = 256
# for i in tqdm(range(0, len(documents), BATCH_SIZE), desc="Processing Batches"):
#     batch = documents[i:i + BATCH_SIZE]
#     vectorstore.add_texts(
#         texts=[doc["content"] for doc in batch],
#         metadatas=[doc["metadata"] for doc in batch],
#         ids=[doc["id"] for doc in batch]
#     )

# # Persist to disk
# vectorstore.persist()
# print("Vector embeddings created and stored successfully!")

C:\Users\aravi\AppData\Local\Temp\ipykernel_29596\3199588759.py:13: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(
C:\Users\aravi\AppData\Local\Temp\ipykernel_29596\3199588759.py:20: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [3]:
# Step 4: Query the vector store to retrieve top-k similar questions
def get_top_k_similar_questions(query, top_k=5):
    results = vectorstore.similarity_search(query, k=top_k)
    return [
        {
            "question": result.page_content,
            "solution": result.metadata["answer"]
        }
        for result in results
    ]

In [4]:
def get_question_prompt(question):
    prefix = 'Please solve the following problem and put your answer at the end with "The answer is: ".\n\n'
    return prefix + question

In [5]:
# Print the texts in the vecotrstore
print("Querying the vector store...")
question = '"What is the value of 2 + 2?"'
query = get_question_prompt(question)
similar_questions = get_top_k_similar_questions(query, top_k=10)
print("Top 5 similar questions:")
for idx, question in enumerate(similar_questions):
    print(f"Question {idx + 1}: {question['question']}")
    print(f"Answer: {question['solution']}\n")
    print("---------------------------------------------------")

Querying the vector store...
Top 5 similar questions:
Question 1: Please solve the following problem and put your answer at the end with "The answer is: ".

What is $\frac{2}{5}$ divided by 3?


Answer: To divide $\frac{2}{5}$ by 3, we can rewrite it as $\frac{2}{5}\div\frac{3}{1}$.
Dividing by a fraction is the same as multiplying by its reciprocal, so we have $\frac{2}{5}\cdot\frac{1}{3}$.
Multiplying the numerators and denominators, we get $\frac{2\cdot1}{5\cdot3}$.
Simplifying, we have $\frac{2}{15}$.
Therefore, $\frac{2}{5}$ divided by 3 is $\boxed{\frac{2}{15}}$.
The answer is: \frac{2}{15}

---------------------------------------------------
Question 2: Please solve the following problem and put your answer at the end with "The answer is: ".

What is $\frac{2}{5}$ divided by 3?


Answer: To divide a fraction by a whole number, you can multiply the denominator of the fraction by the whole number. Alternatively, you can think of it as multiplying the fraction by the reciprocal of 

In [6]:
similar_questions[:3]

[{'question': 'Please solve the following problem and put your answer at the end with "The answer is: ".\n\nWhat is $\\frac{2}{5}$ divided by 3?\n\n',
  'solution': 'To divide $\\frac{2}{5}$ by 3, we can rewrite it as $\\frac{2}{5}\\div\\frac{3}{1}$.\nDividing by a fraction is the same as multiplying by its reciprocal, so we have $\\frac{2}{5}\\cdot\\frac{1}{3}$.\nMultiplying the numerators and denominators, we get $\\frac{2\\cdot1}{5\\cdot3}$.\nSimplifying, we have $\\frac{2}{15}$.\nTherefore, $\\frac{2}{5}$ divided by 3 is $\\boxed{\\frac{2}{15}}$.\nThe answer is: \\frac{2}{15}'},
 {'question': 'Please solve the following problem and put your answer at the end with "The answer is: ".\n\nWhat is $\\frac{2}{5}$ divided by 3?\n\n',
  'solution': "To divide a fraction by a whole number, you can multiply the denominator of the fraction by the whole number. Alternatively, you can think of it as multiplying the fraction by the reciprocal of the whole number. Let's use the second method:\n\n